In [2]:
# Libraries Import #

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import Counter
import pandas as pd
import seaborn as sns
from PIL import Image
import hashlib
%matplotlib inline

In [13]:
# Project directory definition #

project_dir = os.path.abspath('').rsplit(os.path.sep, 1)[0].replace('\\', '/')

In [ ]:
# Uploading and modifying the table with age assignment from the initial set #

labels_dir = project_dir + '/data/external/labels.xlsx'
labels1 = pd.read_excel(labels_dir)
labels_age = labels1.loc[labels1.age.isin(['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']), :]

In [ ]:
# Printing the age table for images from the initial set #

labels_age

In [ ]:
# Generating bar chart of age category frequencies in the initial set #

fig1_dir = project_dir + '/Figures/figure1.jpg'
plt.figure(figsize=[8, 5])
splot = sns.countplot(labels_age['age'], order=['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)'])
plt.xlabel('wiek', fontsize=12)
plt.ylabel('liczebność', fontsize=12)
for p in splot.patches:
    splot.annotate(p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 5), textcoords = 'offset points')
plt.savefig(fig1_dir)

In [ ]:
# Displaying the number of photos not assigned to any age categories #

labels1.loc[~labels1.age.isin(['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']), :].age.value_counts()

In [ ]:
# Generating gender distribution bar chart for data in the initial set #

fig2_dir = project_dir + '/Figures/figure2.jpg'
fig, ax = plt.subplots(figsize=(8, 5))

splot = sns.countplot(labels1['gender'])
labels = [item.get_text() for item in ax.get_xticklabels()]
labels[0] = 'Kobiety'
labels[1] = 'Mężczyźni'
labels[2] = 'Płeć nieznana'
ax.set_xticklabels(labels)
ax.set_xlabel('Płeć', fontsize=12)
ax.set_ylabel('Liczebność', fontsize=12)

for p in splot.patches:
    splot.annotate(p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 5), textcoords = 'offset points')

plt.savefig(fig2_dir)

In [ ]:
# Generating gender distribution bar chart depending on the age category for data in the initial set #

fig3_dir = project_dir + '/Figures/figure3.jpg'
plt.figure(figsize=(8, 5))
sns.countplot(x='age',hue='gender',data=labels_age, order=['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)'])
plt.xlabel('wiek', fontsize=12)
plt.ylabel('liczebność', fontsize=12)
plt.legend(['Kobiety', 'Mężczyźni', 'Płeć nieznana'])

plt.savefig(fig3_dir)

In [ ]:
# Generating gender distribution bar chart depending on the age category for data in the initial set #

fig4_dir = project_dir + '/Figures/figure4.jpg'
age_category = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']

fig, ax = plt.subplots(4, 2, figsize=(11, 23))

i = 0
j = 0
for category in age_category:
    counts = labels_age.loc[labels_age.age.isin([category]), 'gender'].value_counts()
    female = list(counts.index).index('f')
    male = list(counts.index).index('m')
    if len(list(counts.index)) == 3:
        unknown = list(counts.index).index('u')
        counts_new = pd.Series(data=[counts[female], counts[male], counts[unknown]], index=['f', 'm', 'u'])
    else:
        counts_new = pd.Series(data=[counts[female], counts[male]], index=['f', 'm'])
    barplot = ax[i, j].bar(counts_new.index, counts_new.values)
    labels = [item.get_text() for item in ax[i, j].get_xticklabels()]
    labels[0] = 'Kobiety'
    barplot[0].set_color('blueviolet')
    labels[1] = 'Mężczyźni'
    barplot[1].set_color('darkorange')
    if len(list(counts_new.index)) == 3:
        labels[2] = 'Płeć nieznana'
        barplot[2].set_color('forestgreen')
    ax[i, j].set_xticklabels(labels)
    ax[i, j].set_xlabel('Płeć', fontsize=12)
    ax[i, j].set_ylabel('Liczebność', fontsize=12)
    ax[i, j].set_title("Kategoria '" + category + "'")
    
    for bar in barplot.patches:
        ax[i, j].annotate(bar.get_height(), (bar.get_x() + bar.get_width() / 2., bar.get_height()), ha = 'center', va = 'center', xytext = (0, 5), textcoords = 'offset points')

 
    j += 1
    if j == 2:
        j = 0
        i += 1
        
plt.savefig(fig4_dir)

In [ ]:
# Uploading and modifying the age table for images #

labels = pd.read_excel(labels_dir)

labels.loc[labels.age == '(0, 2)', 'age'] = '(0, 6)'
labels.loc[labels.age == '(4, 6)', 'age'] = '(0, 6)'
labels.loc[labels.age == '(8, 12)', 'age'] = '(7, 20)'
labels.loc[labels.age == '(15, 20)', 'age'] = '(7, 20)'
labels.loc[labels.age == '(8, 23)', 'age'] = '(7, 20)'
labels.loc[labels.age == '(25, 32)', 'age'] = '(21, 37)'
labels.loc[labels.age == '(38, 43)', 'age'] = '(38, 59)'
labels.loc[labels.age == '(60, 100)', 'age'] = '(60+)'
labels.loc[labels.age == '(48, 53)', 'age'] = '(38, 59)'
labels.loc[labels.age == '(38, 48)', 'age'] = '(38, 59)'
labels.loc[labels.age == 2, 'age'] = '(0, 6)'
labels.loc[labels.age == 3, 'age'] = '(0, 6)'
labels.loc[labels.age == 13, 'age'] = '(7, 20)'
labels.loc[labels.age == 22, 'age'] = '(21, 37)'
labels.loc[labels.age == 23, 'age'] = '(21, 37)'
labels.loc[labels.age == 29, 'age'] = '(21, 37)'
labels.loc[labels.age == 32, 'age'] = '(21, 37)'
labels.loc[labels.age == 33, 'age'] = '(21, 37)'
labels.loc[labels.age == 34, 'age'] = '(21, 37)'
labels.loc[labels.age == 35, 'age'] = '(21, 37)'
labels.loc[labels.age == 36, 'age'] = '(21, 37)'
labels.loc[labels.age == 42, 'age'] = '(38, 59)'
labels.loc[labels.age == 45, 'age'] = '(38, 59)'
labels.loc[labels.age == 46, 'age'] = '(38, 59)'
labels.loc[labels.age == 55, 'age'] = '(38, 59)'
labels.loc[labels.age == 56, 'age'] = '(38, 59)'
labels.loc[labels.age == 57, 'age'] = '(38, 59)'
labels.loc[labels.age == 58, 'age'] = '(38, 59)'


labels.loc[labels.age == '(0, 6)', 'age'] = 0
labels.loc[labels.age == '(7, 20)', 'age'] = 1
labels.loc[labels.age == '(21, 37)', 'age'] = 2
labels.loc[labels.age == '(38, 59)', 'age'] = 3
labels.loc[labels.age == '(60+)', 'age'] = 4

In [ ]:
# Printing the age table #

labels

In [ ]:
# Controling the number of age categories in the dataset #

print('Number of images in the dataset:', len(np.unique(labels.index)))
print('Age categories frequencies in the dataset:', labels['age'].value_counts(), sep='\n')

In [ ]:
# Bar chart of the age categories frequencies #

print(sns.countplot(labels['age']))

In [ ]:
# Uploading photos from the initial repository 'external' and saving them in folder 'raw' #

file_dir = project_dir + '/data/external'
file_dir1 = project_dir + '/data/raw'
filenames = os.listdir(file_dir)
images_number = len(np.unique(labels.index)) # number of images in the initial file
w = 0
for filename in filenames:
    images = os.listdir(file_dir + '/' + filename)
    for i in images:
        if i.endswith('.jpg'):
            image_name = i.split(sep='.', maxsplit=1)[1]

            img_array = plt.imread(os.path.join(file_dir, filename, i))
            
            im = Image.fromarray(img_array)
            im.save(file_dir1 + '/' + filename + '_' + image_name)
            
            if w > 5:
                if w % np.round(0.1 * images_number, 0) == 0:
                    progress = np.round(w / images_number, 1) * 100
                    print('Progress: ' + str(progress) + '%')
                    
            w += 1
            
            if w == images_number:
                print('Progress: 100.0%')

In [ ]:
# Checking if the photos are not corrupted - displaying the number of damaged photos in the repository #

images = os.listdir(file_dir1)
corrupted_images = []
w = 0
for image in images:
    
    try:
      img = Image.open(os.path.join(file_dir1, image))
      img.verify()
    except (IOError, SyntaxError):
      corrupted_images.append(image)
    finally:
        if w > 5:
            if w % np.round(0.1 * images_number, 0) == 0:
                progress = np.round(w / images_number, 1) * 100
                print('Progress: ' + str(progress) + '%')

        w += 1

        if w == images_number:
            print('Progress: 100.0%')
    
print('Corrupted images number:', len(corrupted_images))

In [ ]:
# Removing duplicate images - throwing duplicate names into the list #

duplicates = []
hash_keys = {}
w = 0

for index, filename in enumerate(os.listdir(file_dir1)):
    with open(os.path.join(file_dir1, filename), 'rb') as f:
        filehash = hashlib.md5(f.read()).hexdigest()
        if filehash not in hash_keys:
            hash_keys[filehash] = index
            
        else:
            duplicates.append(filename)
            
        if w > 5:
            if w % np.round(0.1 * images_number, 0) == 0:
                progress = np.round(w / images_number, 1) * 100
                print('Progress: ' + str(progress) + '%')
                
        w += 1
        
        if w == images_number:
            print('Progress: 100.0%')

print('Duplicates number in the initial dataset:', len(duplicates))

In [ ]:
# Removing duplicate photos from the age table #

labels_new = labels.loc[~labels.original_image_full_name.isin(duplicates), :]

In [ ]:
# Printing the age table after removing duplicates #

labels_new

In [ ]:
# Displaying age categories frequencies after removing duplicates #

labels_new.age.value_counts()

In [ ]:
# Displaying genders frequencies in age categories after removing duplicates #

labels_new1 = labels_new.groupby('age')
labels_new1.gender.value_counts()

In [ ]:
# Uploading images from the repository to check their resolution #

images = os.listdir(file_dir1)
dataset_resolution = []
w = 0
for i in images:     
    if w > 5:
        if w % np.round(0.1 * images_number, 0) == 0:
            progress = np.round(w / images_number, 1) * 100
            print('Progress: ' + str(progress) + '%')
    w += 1

    if w == images_number:
        print('Progress: 100.0%')

    if i in duplicates:
        continue

    img_array = plt.imread(os.path.join(file_dir1, i))
    dataset_resolution.append(img_array)

In [ ]:
# Generating a graph with image resolutions and selecting the appropriate resolution for resizing #

fig5_dir = project_dir + '/Figures/figure5.jpg'
heights = []
widths = []
ratio = []
for image in dataset_resolution:
    height = image.shape[0]
    width = image.shape[1]
    aspect_ratio = round(image.shape[1] / image.shape[0], 2) 
    heights.append(height)
    widths.append(width)
    ratio.append(aspect_ratio)
    
resolution_table = pd.DataFrame({'height': heights, 'width': widths, 'aspect_ratio': ratio})

fig, ax = plt.subplots(figsize=(10, 8))

ax.scatter(resolution_table.width, resolution_table.height, color='blue', alpha=0.5, s=resolution_table.aspect_ratio*100, label='Format konkretnego zdjęcia')
x = np.linspace(*ax.get_xlim())
ax.plot(x, x, 'red', linewidth=2, label='Format obrazu (Aspect ratio) = 1')
ax.set_title('Rozdzielczość zdjęć', fontsize=16)
ax.set_xlabel('Szerokość', fontsize=12)
ax.set_ylabel('Wysokość', fontsize=12)
ax.legend(loc='upper left')
plt.savefig(fig5_dir)

In [ ]:
# Generating image examples of using the zero padding method #

fig6_dir = project_dir + '/Figures/figure6.jpg'
img_array1 = dataset_resolution[202]
img_array2 = dataset_resolution[1111]

old_image_height1, old_image_width1, channels1 = img_array1.shape
old_image_height2, old_image_width2, channels2 = img_array2.shape

if old_image_height1 != old_image_width1:

    new_image_width1 = max(old_image_height1, old_image_width1)
    new_image_height1 = max(old_image_height1, old_image_width1)
    color1 = (0,0,0)
    result1 = np.full((new_image_height1, new_image_width1, 3), color1, dtype=np.uint8)

    x_center1 = (new_image_width1 - old_image_width1) // 2
    y_center1 = (new_image_height1 - old_image_height1) // 2

    result1[y_center1:y_center1+old_image_height1, 
           x_center1:x_center1+old_image_width1] = img_array1

else:

    result1 = img_array1
    
if old_image_height2 != old_image_width2:

    new_image_width2 = max(old_image_height2, old_image_width2)
    new_image_height2 = max(old_image_height2, old_image_width2)
    color2 = (0,0,0)
    result2 = np.full((new_image_height2, new_image_width2, 3), color2, dtype=np.uint8)

    x_center2 = (new_image_width2 - old_image_width2) // 2
    y_center2 = (new_image_height2 - old_image_height2) // 2

    result2[y_center2:y_center2+old_image_height2, 
           x_center2:x_center2+old_image_width2] = img_array2

else:

    result2 = img_array2

plt.figure(figsize=[8, 5])

plt.subplot(2, 2, 1)
plt.imshow(img_array1)
plt.title('Zdjęcie oryginalne')

plt.subplot(2, 2, 2)
plt.imshow(result1)
plt.title('Zdjęcie po modyfikacji')
    
plt.subplot(2, 2, 3)
plt.imshow(img_array2)

plt.subplot(2, 2, 4)
plt.imshow(result2)
plt.savefig(fig6_dir)

In [ ]:
# Uploading images from the repository, padding and resizing them  #

images = os.listdir(file_dir1)
dataset = []
w = 0
    
for i in images:
    if w > 5:
        if w % np.round(0.1 * images_number, 0) == 0:
            progress = np.round(w / images_number, 1) * 100
            print('Progress: ' + str(progress) + '%')
    w += 1
    
    if w == images_number:
        print('Progress: 100.0%')
    
    if i in duplicates:
        continue

    try:
        label_y = labels_new.loc[labels_new.original_image_full_name == i, 'age'].iloc[0]
    except:
        continue
    else:

        img_array = plt.imread(os.path.join(file_dir1, i))

        old_image_height, old_image_width, channels = img_array.shape

        if old_image_height != old_image_width:

            new_image_width = max(old_image_height, old_image_width)
            new_image_height = max(old_image_height, old_image_width)
            color = (0,0,0)
            result = np.full((new_image_height, new_image_width, 3), color, dtype=np.uint8)

            x_center = (new_image_width - old_image_width) // 2
            y_center = (new_image_height - old_image_height) // 2

            result[y_center:y_center+old_image_height, 
                   x_center:x_center+old_image_width] = img_array

        else:

            result = img_array

        img_array = np.array(Image.fromarray(result).resize((256, 256)))

        dataset.append([img_array, label_y])

In [ ]:
# Printing the number of images after resizing size #

print(len(dataset))

In [ ]:
# Saving photos to folders with age categories in the repository for quality control of assigning photos to categories #

file_dir2 = project_dir + '/data/interim'
image_number_new = len(dataset)
w = 0
for z in range(5):
    for i in range(len(dataset)):
        if dataset[i][1] == z:
            im = Image.fromarray(dataset[i][0])
            im.save(file_dir2 + '/Category_' + str(z) + '/zdjecie_' + str(i) + '.jpeg')
            
            if w > 5:
                if w % np.round(0.1 * image_number_new, 0) == 0:
                    progress = np.round(w / image_number_new, 1) * 100
                    print('Progress: ' + str(progress) + '%')
            w += 1

            if w == image_number_new:
                print('Progress: 100.0%')

In [ ]:
# Image examples from each category:
# - 0 -> Infant and preschool children (0-6 years)
# - 1 -> Early school age children and adolescents (7-20 years)
# - 2 -> Adults in the mobile working age (21-37 years)
# - 3 -> Middle aged adults (38-59 years)
# - 4 -> Adults over 60 (seniors)

fig7_dir = project_dir + '/Figures/figure7.jpg'
plt.figure(figsize=[15, 15])

Category_0 = list(zip ([1, 6, 11, 16, 21], [804, 790, 2971, 6264, 8351]))
Category_1 = list(zip ([2, 7, 12, 17, 22], [2097, 3569, 4138, 12363, 13466]))
Category_2 = list(zip ([3, 8, 13, 18, 23], [642, 10, 3238, 692, 2371]))
Category_3 = list(zip ([4, 9, 14, 19, 24], [15178, 11936, 458, 1768, 18275]))
Category_4 = list(zip ([5, 10, 15, 20, 25], [17687, 18298, 12506, 5241, 17962]))

for i in Category_0:
    plt.subplot(5, 5, i[0])
    plt.imshow(dataset[i[1]][0])
    if i[0] == 1:
        plt.title('Kategoria 0')
   
for i in Category_1:
    plt.subplot(5, 5, i[0])
    plt.imshow(dataset[i[1]][0])
    if i[0] == 2:
        plt.title('Kategoria 1')
    
for i in Category_2:
    plt.subplot(5, 5, i[0])
    plt.imshow(dataset[i[1]][0])
    if i[0] == 3:
        plt.title('Kategoria 2')

for i in Category_3:
    plt.subplot(5, 5, i[0])
    plt.imshow(dataset[i[1]][0])
    if i[0] == 4:
        plt.title('Kategoria 3')

for i in Category_4:
    plt.subplot(5, 5, i[0])
    plt.imshow(dataset[i[1]][0])
    if i[0] == 5:
        plt.title('Kategoria 4')
    
plt.savefig(fig7_dir)

In [ ]:
# Saving images and their age categories in separate lists #

feature_list = []
label_list = []
for feature, label in dataset:
    feature_list.append(feature)
    label_list.append(label)

In [ ]:
# Saving a list with images and a list with their age categories in a numpy array #

X_dataset = np.array(feature_list)
Y_dataset = np.array(label_list)

In [ ]:
# Printing the numpy arrays dimensions respectively for the image set and for their age category #

print(X_dataset.shape)
print(Y_dataset.shape)

In [ ]:
# Division of the set into training and test sets #

train_X, val_X, train_Y, val_Y = train_test_split(X_dataset, Y_dataset, test_size=0.4, shuffle=True, stratify=Y_dataset)

In [ ]:
# Division of the test set into validation and test sets #

validation_X, test_X, validation_Y, test_Y = train_test_split(val_X, val_Y, test_size=0.5, shuffle=True, stratify=val_Y)

In [ ]:
# Printing the dimensions of the created sets #

print('Training set X:', train_X.shape)
print('Training set Y:', train_Y.shape)
print('Validation set X:', validation_X.shape)
print('Validation set Y:', validation_Y.shape)
print('Test set X:', test_X.shape)
print('Test set Y:', test_Y.shape)

In [ ]:
# Saving training, validation and test sets with their age categories in the 'processed' repository #

file_dir3 = project_dir + '/data/processed'
names_train = []
names_validation = []
names_test = []
w = 0
for i in range(train_X.shape[0]):
    w += 1
    im = Image.fromarray(train_X[i].astype(np.uint8))
    im.save(file_dir3 + '/train/photos/zdjecie_' + str(w) + '_kategoria_' + str(train_Y[i]) + '.jpg')
    image_name = 'zdjecie_' + str(w)
    names_train.append(image_name)

table1 = pd.DataFrame({'original_image': names_train, 'labels': train_Y})
table1.to_csv(file_dir3 + '/train/labels.csv', index=False)
print('Training set saved!')

for i in range(validation_X.shape[0]):
    w += 1
    im = Image.fromarray(validation_X[i].astype(np.uint8))
    im.save(file_dir3 + '/validation/photos/zdjecie_' + str(w) + '_kategoria_' + str(validation_Y[i]) +'.jpg')
    image_name = 'zdjecie_' + str(w)
    names_validation.append(image_name)
    
table2 = pd.DataFrame({'original_image': names_validation, 'labels': validation_Y})
table2.to_csv(file_dir3 + '/validation/labels.csv', index=False)
print('Validation set saved!')

for i in range(test_X.shape[0]):
    w += 1
    im = Image.fromarray(test_X[i].astype(np.uint8))
    im.save(file_dir3 + '/test/photos/zdjecie_' + str(w) + '_kategoria_' + str(test_Y[i]) +'.jpg')
    image_name = 'zdjecie_' + str(w)
    names_test.append(image_name)
                       
table3 = pd.DataFrame({'original_image': names_test, 'labels': test_Y})
table3.to_csv(file_dir3 + '/test/labels.csv', index=False)
print('Test set saved!')

In [ ]:
# Reading images into training, validation and test sets from the repository #

filenames = os.listdir(file_dir3)
for filename in filenames:
    dataset_import = []
    category_list = []
    images = os.listdir(file_dir3 + '/' + filename + '/photos')
    for image in images:
        img_array = plt.imread(os.path.join(file_dir3, filename, 'photos', image))
        category = int(image.split('_', 3)[3].split('.', 1)[0])
        dataset_import.append(img_array)
        category_list.append(category)
    if filename == 'train':
        train_X = np.array(dataset_import)
        train_Y = np.array(category_list)
    if filename == 'validation':
        validation_X = np.array(dataset_import)
        validation_Y = np.array(category_list)
    if filename == 'test':
        test_X = np.array(dataset_import)
        test_Y = np.array(category_list)
    print('Uploaded set ', filename, '!', sep='')

In [ ]:
# Printing age categories in training, validation and test sets #

print('Age categories frequencies in the training set:', Counter(train_Y))
print('Age categories frequencies in the validation set:', Counter(validation_Y))
print('Age categories frequencies in the test set:', Counter(test_Y))

In [ ]:
# Normalization of pixel values in images from training, validation and test sets #

train_X = train_X / 255.0
validation_X = validation_X / 255.0
test_X = test_X / 255.0

In [15]:
# Model I filepath #

model_filepath1 = project_dir + '/models/model_checkpoint1'

In [ ]:
# Model I #

# Model initialization #

classifier1 = Sequential()

# Model architecture #

classifier1.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1.add(Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1.add(Flatten())
classifier1.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath1, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model architecture summary I #

classifier1.summary()

In [ ]:
# Model training I #

model_plain1 = classifier1.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy I - the best trained on the validation set #

Model_I = load_model(model_filepath1)
Model_I.evaluate(x=validation_X, y=validation_Y)

In [18]:
# Model Ia filepath #

model_filepath1a = project_dir + '/models/model_checkpoint1a'

In [ ]:
# Model Ia #

# Model initialization #

classifier1a = Sequential()

# Model architecture #

classifier1a.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1a.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1a.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1a.add(Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1a.add(Flatten())
classifier1a.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier1a.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath1a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training Ia #

model_plain1a = classifier1.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy Ia - the best trained on the validation set #

Model_Ia = load_model(model_filepath1a)
Model_Ia.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model Ib filepath #

model_filepath1b = project_dir + '/models/model_checkpoint1b'

In [ ]:
# Model Ib #

# Model initialization #

classifier1b = Sequential()

# Model architecture #

classifier1b.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1b.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1b.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1b.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1b.add(Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier1b.add(Flatten())
classifier1b.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1b.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier1b.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier1b.compile(optimizer=optimizers.Adam(0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath1b, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training Ib #

model_plain1a = classifier1.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy Ib - the best trained on the validation set #

Model_Ib = load_model(model_filepath1b)
Model_Ib.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model II filepath #

model_filepath2 = project_dir + '/models/model_checkpoint2'

In [ ]:
# Model II #

# Model initialization #

classifier2 = Sequential()

# Model architecture #

classifier2.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2.add(Flatten())
classifier2.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath2, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training II #

model_plain2 = classifier2.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy II - the best trained on the validation set #

Model_II = load_model(model_filepath2)
Model_II.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model IIa filepath #

model_filepath2a = project_dir + '/models/model_checkpoint2a'

In [ ]:
# Model IIa #

# Model initialization #

classifier2a = Sequential()

# Model architecture #

classifier2a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2a.add(Flatten())
classifier2a.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier2a.compile(optimizer=optimizers.Adam(0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath2a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training IIa #

model_plain2a = classifier2a.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy IIa - the best trained on the validation set #

Model_IIa = load_model(model_filepath2a)
Model_IIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model IIb filepath #

model_filepath2b = project_dir + '/models/model_checkpoint2b'

In [ ]:
# Model IIb #

# Model initialization #

classifier2b = Sequential()

# Model architecture #

classifier2b.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2b.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2b.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier2b.add(Flatten())
classifier2b.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2b.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier2b.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier2b.compile(optimizer=optimizers.Adam(0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath2b, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training IIb #

model_plain2b = classifier2a.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy IIb - the best trained on the validation set #

Model_IIb = load_model(model_filepath2b)
Model_IIb.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model III filepath #

model_filepath3 = project_dir + '/models/model_checkpoint3'

In [ ]:
# Model III #

# Model initialization #

classifier3 = Sequential()

# Model architecture #

classifier3.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier3.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier3.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier3.add(Flatten())
classifier3.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath3, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier3.summary()

In [ ]:
# Model training III - poprawione validation #

model_plain3 = classifier3.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy III - the best trained on the validation set #

Model_III = load_model(model_filepath3)
Model_III.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model IIIa filepath #

model_filepath3a = project_dir + '/models/model_checkpoint3a'

In [ ]:
# Model IIIa #

# Model initialization #

classifier3a = Sequential()

# Model architecture #

classifier3a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier3a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier3a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier3a.add(Flatten())
classifier3a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier3a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier3a.compile(optimizer=optimizers.Adam(0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath3, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training IIIa #

model_plain3a = classifier3a.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy IIIa - the best trained on the validation set #

Model_IIIa = load_model(model_filepath3)
Model_IIIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model IV filepath #

model_filepath4 = project_dir + '/models/model_checkpoint4'

In [ ]:
# Model IV #

# Model initialization #

classifier4 = Sequential()

# Model architecture #

classifier4.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4.add(Flatten())
classifier4.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath4, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier4.summary()

In [ ]:
# Model training IV #

model_plain4 = classifier4.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy IV - the best trained on the validation set #

Model_IV = load_model(model_filepath4)
Model_IV.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model IVa filepath #

model_filepath4a = project_dir + '/models/model_checkpoint4a'

In [ ]:
# Model IVa #

# Model initialization #

classifier4a = Sequential()

# Model architecture #

classifier4a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4a.add(Flatten())
classifier4a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier4a.compile(optimizer=optimizers.Adam(0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath4a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training IVa #

model_plain4a = classifier4a.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy IVa - the best trained on the validation set #

Model_IVa = load_model(model_filepath4a)
Model_IVa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model IVb filepath #

model_filepath4b = project_dir + '/models/model_checkpoint4b'

In [ ]:
# Model IVb #

# Model initialization #

classifier4b = Sequential()

# Model architecture #

classifier4b.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4b.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4b.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier4b.add(Flatten())
classifier4b.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4b.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier4b.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier4b.compile(optimizer=optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath4b, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training IVb #

model_plain4b = classifier4b.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy IVb - the best trained on the validation set #

Model_IVb = load_model(model_filepath4b)
Model_IVb.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model V filepath #

model_filepath5 = project_dir + '/models/model_checkpoint5'

In [ ]:
# Model V #

# Model initialization #

classifier5 = Sequential()

# Model architecture #

classifier5.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5.add(Flatten())
classifier5.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath5, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier5.summary()

In [ ]:
# Model training V #

model_plain5 = classifier5.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy V - the best trained on the validation set #

Model_V = load_model(model_filepath5)
Model_V.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model Va filepath #

model_filepath5a = project_dir + '/models/model_checkpoint5a'

In [ ]:
# Model Va #

# Model initialization #

classifier5a = Sequential()

# Model architecture #

classifier5a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5a.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier5a.add(Flatten())
classifier5a.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier5a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier5a.compile(optimizer=optimizers.Adam(0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath5a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training Va #

model_plain5a = classifier5a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy Va - the best trained on the validation set #

Model_Va = load_model(model_filepath5a)
Model_Va.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model VI filepath #

model_filepath6 = project_dir + '/models/model_checkpoint6'

In [ ]:
# Model VI #

# Model initialization #

classifier6 = Sequential()

# Model architecture #

classifier6.add(Conv2D(16, (7, 7), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6.add(Conv2D(16, (5, 5), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6.add(Flatten())
classifier6.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier6.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath6, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier6.summary()

In [ ]:
# Model training VI #

model_plain6 = classifier6.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy VI - the best trained on the validation set #

Model_VI = load_model(model_filepath6)
Model_VI.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model VIa filepath #

model_filepath6a = project_dir + '/models/model_checkpoint6a'

In [ ]:
# Model VIa #

# Model initialization #

classifier6a = Sequential()

# Model architecture #

classifier6a.add(Conv2D(16, (7, 7), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6a.add(Conv2D(16, (5, 5), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6a.add(Flatten())
classifier6a.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier6a.compile(optimizer=optimizers(0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath6a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training VIa #

model_plain6a = classifier6a.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy VIa - the best trained on the validation set #

Model_VIa = load_model(model_filepath6a)
Model_VIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model VIb filepath #

model_filepath6b = project_dir + '/models/model_checkpoint6b'

In [ ]:
# Model VIb #

# Model initialization #

classifier6b = Sequential()

# Model architecture #

classifier6b.add(Conv2D(16, (7, 7), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6b.add(Conv2D(16, (5, 5), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6b.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier6b.add(Flatten())
classifier6b.add(Dense(units=512, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6b.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier6b.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier6b.compile(optimizer=optimizers(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath6b, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training VIb #

model_plain6b = classifier6b.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy VIb - the best trained on the validation set #

Model_VIb = load_model(model_filepath6b)
Model_VIb.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model VII filepath #

model_filepath7 = project_dir + '/models/model_checkpoint7'

In [ ]:
# Model VII (double Dropout) #

# Model initialization #

classifier7 = Sequential()

# Model architecture #

classifier7.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7.add(Flatten())
classifier7.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7.add(Dropout(0.5))
classifier7.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7.add(Dropout(0.5))
classifier7.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier7.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath7, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier7.summary()

In [ ]:
# Model training VII #

model_dropout7 = classifier7.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy VII - the best trained on the validation set #

Model_VII = load_model(model_filepath7)
Model_VII.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model VIIa filepath #

model_filepath7a = project_dir + '/models/model_checkpoint7a'

In [ ]:
# Model VIIa (double Dropout) #

# Model initialization #

classifier7a = Sequential()

# Model architecture #

classifier7a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier7a.add(Flatten())
classifier7a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7a.add(Dropout(0.5))
classifier7a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier7a.add(Dropout(0.5))
classifier7a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier7a.compile(optimizer=optimizers.Adam(0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath7a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training VIIa #

model_dropout7a = classifier7a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy VIIa - the best trained on the validation set #

Model_VIIa = load_model(model_filepath7a)
Model_VIIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model VIII filepath #

model_filepath8 = project_dir + '/models/model_checkpoint8'

In [ ]:
# Model VIII (single Dropout) #

# Model initialization #

classifier8 = Sequential()

# Model architecture #

classifier8.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8.add(Flatten())
classifier8.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8.add(Dropout(0.5))
classifier8.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier8.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath8, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier8.summary()

In [ ]:
# Model training VIII #

model_dropout8 = classifier8.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy VIII - the best trained on the validation set #

Model_VIII = load_model(model_filepath8)
Model_VIII.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model VIIIa filepath #

model_filepath8a = project_dir + '/models/model_checkpoint8a'

In [ ]:
# Model VIIIa (single Dropout) #

# Model initialization #

classifier8a = Sequential()

# Model architecture #

classifier8a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier8a.add(Flatten())
classifier8a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8a.add(Dropout(0.5))
classifier8a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier8a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier8a.compile(optimizer=optimizers.Adam(0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath8a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training VIIIa #

model_dropout8a = classifier8a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy VIIIa - the best trained on the validation set #

Model_VIIIa = load_model(model_filepath8a)
Model_VIIIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model IX filepath #

model_filepath9 = project_dir + '/models/model_checkpoint9'

In [ ]:
# Model IX (single Dropout - second dense layer) #

# Model initialization #

classifier9 = Sequential()

# Model architecture #

classifier9.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9.add(Flatten())
classifier9.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9.add(Dropout(0.5))
classifier9.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier9.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath9, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier9.summary()

In [ ]:
# Model training IX #

model_dropout9 = classifier9.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy IX - the best trained on the validation set #

Model_IX = load_model(model_filepath9)
Model_IX.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model IXa filepath #

model_filepath9a = project_dir + '/models/model_checkpoint9a'

In [ ]:
# Model IXa (single Dropout - second dense layer) #

# Model initialization #

classifier9a = Sequential()

# Model architecture #

classifier9a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier9a.add(Flatten())
classifier9a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier9a.add(Dropout(0.5))
classifier9a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier9a.compile(optimizer=optimizers.Adam(0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath9a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training IXa #

model_dropout9a = classifier9a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy IXa - the best trained on the validation set #

Model_IXa = load_model(model_filepath9a)
Model_IXa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model X filepath #

model_filepath10 = project_dir + '/models/model_checkpoint10'

In [ ]:
# Model X (max norm on all layers) #

# Model initialization #

classifier10 = Sequential()

# Model architecture #

classifier10.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10.add(Flatten())
classifier10.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier10.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath10, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier10.summary()

In [ ]:
# Model training X #

model_max_norm10 = classifier10.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy X - the best trained on the validation set #

Model_X = load_model(model_filepath10)
Model_X.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model Xa filepath #

model_filepath10a = project_dir + '/models/model_checkpoint10a'

In [ ]:
# Model Xa (max norm on all layers) #

# Model initialization #

classifier10a = Sequential()

# Model architecture #

classifier10a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier10a.add(Flatten())
classifier10a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier10a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier10a.compile(optimizer=optimizers.Adam(0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath10a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training X #

model_max_norm10a = classifier10a.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy Xa - the best trained on the validation set #

Model_Xa = load_model(model_filepath10a)
Model_Xa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XI filepath #

model_filepath11 = project_dir + '/models/model_checkpoint11'

In [ ]:
# Model XI (max norm only on dense layers) #

# Model initialization #

classifier11 = Sequential()

# Model architecture #

classifier11.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11.add(Flatten())
classifier11.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier11.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier11.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier11.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath11, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier11.summary()

In [ ]:
# Model training XI #

model_max_norm11 = classifier11.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XI - the best trained on the validation set #

Model_XI = load_model(model_filepath11)
Model_XI.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XIa filepath #

model_filepath11a = project_dir + '/models/model_checkpoint11a'

In [ ]:
# Model XIa (max norm only on dense layers) #

# Model initialization #

classifier11a = Sequential()

# Model architecture #

classifier11a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11a.add(Flatten())
classifier11a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier11a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier11a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier11a.compile(optimizer=optimizers.Adam(0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath11a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XIa #

model_max_norm11a = classifier11a.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XIa - the best trained on the validation set #

Model_XIa = load_model(model_filepath11a)
Model_XIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XIb filepath #

model_filepath11b = project_dir + '/models/model_checkpoint11b'

In [ ]:
# Model XIb (max norm only on dense layers) #

# Model initialization #

classifier11b = Sequential()

# Model architecture #

classifier11b.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11b.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11b.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier11b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier11b.add(Flatten())
classifier11b.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier11b.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier11b.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier11b.compile(optimizer=optimizers.Adam(0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath11b, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XIb #

model_max_norm11b = classifier11b.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XIb - the best trained on the validation set #

Model_XIb = load_model(model_filepath11b)
Model_XIb.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XII filepath #

model_filepath12 = project_dir + '/models/model_checkpoint12'

In [ ]:
# Model XII (max norm only on convolutional layers) #

# Model initialization #

classifier12 = Sequential()

# Model architecture #

classifier12.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12.add(Flatten())
classifier12.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier12.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier12.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier12.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath12, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier12.summary()

In [ ]:
# Model training XII #

model_max_norm12 = classifier12.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XII - the best trained on the validation set #

Model_XII = load_model(model_filepath12)
Model_XII.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XIIa filepath #

model_filepath12a = project_dir + '/models/model_checkpoint12a'

In [ ]:
# Model XIIa (max norm only on convolutional layers) #

# Model initialization #

classifier12a = Sequential()

# Model architecture #

classifier12a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier12a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier12a.add(Flatten())
classifier12a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier12a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier12a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier12a.compile(optimizer=optimziers.Adam(0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath12a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XIIa #

model_max_norm12a = classifier12a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XIIa - the best trained on the validation set #

Model_XIIa = load_model(model_filepath12a)
Model_XIIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XIII filepath #

model_filepath13 = project_dir + '/models/model_checkpoint13'

In [ ]:
# Model XIII (max norm only on dense layers + Dropout on two dense layers) #

# Model initialization #

classifier13 = Sequential()

# Model architecture #

classifier13.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13.add(Flatten())
classifier13.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier13.add(Dropout(0.5))
classifier13.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier13.add(Dropout(0.5))
classifier13.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier13.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath13, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier13.summary()

In [ ]:
# Model training XIII #

model_max_norm13 = classifier13.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XIII - the best trained on the validation set #

Model_XIII = load_model(model_filepath13)
Model_XIII.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XIIIa filepath #

model_filepath13a = project_dir + '/models/model_checkpoint13a'

In [ ]:
# Model XIIIa (max norm only on dense layers + Dropout on two dense layers) #

# Model initialization #

classifier13a = Sequential()

# Model architecture #

classifier13a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier13a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier13a.add(Flatten())
classifier13a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier13a.add(Dropout(0.5))
classifier13a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier13a.add(Dropout(0.5))
classifier13a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier13a.compile(optimizer=optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath13a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XIIIa #

model_max_norm13a = classifier13a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XIIIa - the best trained on the validation set #

Model_XIIIa = load_model(model_filepath13a)
Model_XIIIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XIV filepath #

model_filepath14 = project_dir + '/models/model_checkpoint14'

In [ ]:
# Model XIV (max norm only on convolutional layers + Dropout on two dense layers) #

# Model initialization #

classifier14 = Sequential()

# Model architecture #

classifier14.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14.add(Flatten())
classifier14.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier14.add(Dropout(0.5))
classifier14.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier14.add(Dropout(0.5))
classifier14.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier14.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath14, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier14.summary()

In [ ]:
# Model training XIV #

model_max_norm14 = classifier14.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XIV - the best trained on the validation set #

Model_XIV = load_model(model_filepath14)
Model_XIV.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XIVa filepath #

model_filepath14a = project_dir + '/models/model_checkpoint14a'

In [ ]:
# Model XIVa (max norm only on convolutional layers + Dropout on two dense layers) #

# Model initialization #

classifier14a = Sequential()

# Model architecture #

classifier14a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier14a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier14a.add(Flatten())
classifier14a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier14a.add(Dropout(0.5))
classifier14a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier14a.add(Dropout(0.5))
classifier14a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier14a.compile(optimizer=optimizers.Adam(0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath14a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XIVa #

model_max_norm14a = classifier14a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XIVa - the best trained on the validation set #

Model_XIVa = load_model(model_filepath14a)
Model_XIVa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XV filepath #

model_filepath15 = project_dir + '/models/model_checkpoint15'

In [ ]:
# Model XV (max norm on all layers + Dropout on two dense layers) #

# Model initialization #

classifier15 = Sequential()

# Model architecture #

classifier15.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15.add(Flatten())
classifier15.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15.add(Dropout(0.5))
classifier15.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15.add(Dropout(0.5))
classifier15.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier15.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath15, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier15.summary()

In [ ]:
# Model training XV #

model_max_norm15 = classifier15.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XV - the best trained on the validation set #

Model_XV = load_model(model_filepath15)
Model_XV.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XVa filepath #

model_filepath15a = project_dir + '/models/model_checkpoint15a'

In [ ]:
# Model XVa (max norm on all layers + Dropout on two dense layers) #

# Model initialization #

classifier15a = Sequential()

# Model architecture #

classifier15a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15a.add(Flatten())
classifier15a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15a.add(Dropout(0.5))
classifier15a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15a.add(Dropout(0.5))
classifier15a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier15a.compile(optimizer=optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath15a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XVa #

model_max_norm15a = classifier15a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XV - the best trained on the validation set #

Model_XVa = load_model(model_filepath15a)
Model_XVa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XVb filepath #

model_filepath15b = project_dir + '/models/model_checkpoint15b'

In [ ]:
# Model XVb (max norm on all layers + Dropout on two dense layers) #

# Model initialization #

classifier15b = Sequential()

# Model architecture #

classifier15b.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15b.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15b.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier15b.add(Flatten())
classifier15b.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15b.add(Dropout(0.5))
classifier15b.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier15b.add(Dropout(0.5))
classifier15b.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier15b.compile(optimizer=optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath15b, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XVb #

model_max_norm15b = classifier15b.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XVb - the best trained on the validation set #

Model_XVb = load_model(model_filepath15b)
Model_XVb.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XVI filepath #

model_filepath16 = project_dir + '/models/model_checkpoint16'

In [ ]:
# Model XVI (max_norm only on dense layers + Dropout only on first dense layer) #

# Model initialization #

classifier16 = Sequential()

# Model architecture #

classifier16.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16.add(Flatten())
classifier16.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier16.add(Dropout(0.5))
classifier16.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier16.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier16.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath16, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier16.summary()

In [ ]:
# Model training XVI #

model_max_norm16 = classifier16.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XVI - the best trained on the validation set #

Model_XVI = load_model(model_filepath16)
Model_XVI.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XVIa filepath #

model_filepath16a = project_dir + '/models/model_checkpoint16a'

In [ ]:
# Model XVIa (max_norm only on dense layers + Dropout only on first dense layer) #

# Model initialization #

classifier16a = Sequential()

# Model architecture #

classifier16a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier16a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier16a.add(Flatten())
classifier16a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier16a.add(Dropout(0.5))
classifier16a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier16a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier16a.compile(optimizer=optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath16a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XVIa #

model_max_norm16a = classifier16a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XVIa - the best trained on the validation set #

Model_XVIa = load_model(model_filepath16a)
Model_XVIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XVII filepath #

model_filepath17 = project_dir + '/models/model_checkpoint17'

In [ ]:
# Model XVII (max_norm only on dense layers + Dropout only on second dense layer) #

# Model initialization #

classifier17 = Sequential()

# Model architecture #

classifier17.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17.add(Flatten())
classifier17.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier17.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier17.add(Dropout(0.5))
classifier17.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier17.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath17, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier17.summary()

In [ ]:
# Model training XVII #

model_max_norm17 = classifier17.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XVII - the best trained on the validation set #

Model_XVII = load_model(model_filepath17)
Model_XVII.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XVIIa filepath #

model_filepath17a = project_dir + '/models/model_checkpoint17a'

In [ ]:
# Model XVIIa (max_norm only on dense layers + Dropout only on second dense layer) #

# Model initialization #

classifier17a = Sequential()

# Model architecture #

classifier17a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier17a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier17a.add(Flatten())
classifier17a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier17a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))
classifier17a.add(Dropout(0.5))
classifier17a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_constraint=max_norm(3)))

# Model compilation #

classifier17a.compile(optimizer=optimizers.Adam(0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath17a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XVIIa #

model_max_norm17a = classifier17a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XVIIa - the best trained on the validation set #

Model_XVIIa = load_model(model_filepath17a)
Model_XVIIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XVIII filepath #

model_filepath18 = project_dir + '/models/model_checkpoint18'

In [ ]:
# Model XVIII Batch Normalization on all layers #

# Model initialization #

classifier18 = Sequential()

# Model architecture #

classifier18.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18.add(Flatten())
classifier18.add(Dense(units=256, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18.add(Dense(units=128, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier18.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath18, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier18.summary()

In [ ]:
# Model training XVIII #

model_batch_norm18 = classifier18.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XVIII - the best trained on the validation set #

Model_XVIII = load_model(model_filepath18)
Model_XVIII.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XVIIIa filepath #

model_filepath18a = project_dir + '/models/model_checkpoint18a'

In [ ]:
# Model XVIIIa Batch Normalization on all layers #

# Model initialization #

classifier18a = Sequential()

# Model architecture #

classifier18a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18a.add(Flatten())
classifier18a.add(Dense(units=256, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18a.add(Dense(units=128, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier18a.compile(optimizer=optimizers(0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath18a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XVIIIa #

model_batch_norm18a = classifier18a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XVIIIa - the best trained on the validation set #

Model_XVIIIa = load_model(model_filepath18a)
Model_XVIIIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XVIIIb filepath #

model_filepath18b = project_dir + '/models/model_checkpoint18b'

In [ ]:
# Model XVIIIb Batch Normalization on all layers #

# Model initialization #

classifier18b = Sequential()

# Model architecture #

classifier18b.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18b.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18b.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18b.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18b.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier18b.add(Flatten())
classifier18b.add(Dense(units=256, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18b.add(Dense(units=128, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier18b.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier18b.compile(optimizer=optimizers(0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath18b, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XVIIIb #

model_batch_norm18b = classifier18b.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XVIIIb - the best trained on the validation set #

Model_XVIIIb = load_model(model_filepath18b)
Model_XVIIIb.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XIX filepath #

model_filepath19 = project_dir + '/models/model_checkpoint19'

In [ ]:
# Model XIX Batch Normalization only on convolutional layers #

# Model initialization #

classifier19 = Sequential()

# Model architecture #

classifier19.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19.add(Flatten())
classifier19.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier19.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier19.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier19.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath19, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier19.summary()

In [ ]:
# Model training XIX #

model_batch_norm19 = classifier19.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XIX - the best trained on the validation set #

Model_XIX = load_model(model_filepath19)
Model_XIX.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XIXa filepath #

model_filepath19a = project_dir + '/models/model_checkpoint19a'

In [ ]:
# Model XIXa Batch Normalization only on convolutional layers #

# Model initialization #

classifier19a = Sequential()

# Model architecture #

classifier19a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=False,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier19a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier19a.add(Flatten())
classifier19a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier19a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier19a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier19a.compile(optimizer=optimizers.Adam(0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath19a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XIXa #

model_batch_norm19a = classifier19a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XIXa - the best trained on the validation set #

Model_XIXa = load_model(model_filepath19a)
Model_XIXa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XX filepath #

model_filepath20 = project_dir + '/models/model_checkpoint20'

In [ ]:
# Model XX Batch Normalization only on dense layers #

# Model initialization #

classifier20 = Sequential()

# Model architecture #

classifier20.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20.add(Flatten())
classifier20.add(Dense(units=256, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier20.add(Dense(units=128, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier20.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier20.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath20, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier20.summary()

In [ ]:
# Model training XX #

model_batch_norm20 = classifier20.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XX - the best trained on the validation set #

Model_XX = load_model(model_filepath20)
Model_XX.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XXa filepath #

model_filepath20a = project_dir + '/models/model_checkpoint20a'

In [ ]:
# Model XXa Batch Normalization only on dense layers #

# Model initialization #

classifier20a = Sequential()

# Model architecture #

classifier20a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier20a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier20a.add(Flatten())
classifier20a.add(Dense(units=256, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier20a.add(Dense(units=128, activation='relu', use_bias=False, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
BatchNormalization()
classifier20a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier20a.compile(optimizer=optimizers.Adam(0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath20a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XXa #

model_batch_norm20a = classifier20a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XXa - the best trained on the validation set #

Model_XXa = load_model(model_filepath20a)
Model_XXa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XXI filepath #

model_filepath21 = project_dir + '/models/model_checkpoint21'

In [ ]:
# Model XXI - L2 regularization on all layers #

# Model initialization #

classifier21 = Sequential()

# Model architecture #

classifier21.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21.add(Flatten())
classifier21.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', 
                      kernel_regularizer=regularizers.l2(0.01)))
classifier21.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform',
                      kernel_regularizer=regularizers.l2(0.01)))
classifier21.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.01)))

# Model compilation #

classifier21.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath21, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier21.summary()

In [ ]:
# Model training XXI #

model_L2_21 = classifier21.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XXI - the best trained on the validation set #

Model_XXI = load_model(model_filepath21)
Model_XXI.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XXIa filepath #

model_filepath21a = project_dir + '/models/model_checkpoint21a'

In [ ]:
# Model XXIa - L2 regularization on all layers #

# Model initialization #

classifier21a = Sequential()

# Model architecture #

classifier21a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier21a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier21a.add(Flatten())
classifier21a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', 
                      kernel_regularizer=regularizers.l2(0.01)))
classifier21a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform',
                      kernel_regularizer=regularizers.l2(0.01)))
classifier21a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.01)))

# Model compilation #

classifier21a.compile(optimizer=optimizers.Adam(0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath21a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XXIa #

model_L2_21a = classifier21a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XXIa - the best trained on the validation set #

Model_XXIa = load_model(model_filepath21a)
Model_XXIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XXII filepath #

model_filepath22 = project_dir + '/models/model_checkpoint22'

In [ ]:
# Model XXII - L2 regularization only on convolutional layers #

# Model initialization #

classifier22 = Sequential()

# Model architecture #

classifier22.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22.add(Flatten())
classifier22.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier22.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier22.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier22.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath22, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier22.summary()

In [ ]:
# Model training XXII #

model_L2_22 = classifier22.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XXII - the best trained on the validation set #

Model_XXII = load_model(model_filepath22)
Model_XXII.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XXIIa filepath #

model_filepath22a = project_dir + '/models/model_checkpoint22a'

In [ ]:
# Model XXIIa - L2 regularization only on convolutional layers #

# Model initialization #

classifier22a = Sequential()

# Model architecture #

classifier22a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.005)))
classifier22a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier22a.add(Flatten())
classifier22a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier22a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier22a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform'))

# Model compilation #

classifier22a.compile(optimizer=optimizers.Adam(0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath22a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XXIIa #

model_L2_2a = classifier22a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XXIIa - the best trained on the validation set #

Model_XXIIa = load_model(model_filepath22a)
Model_XXIIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XXIII filepath #

model_filepath23 = project_dir + '/models/model_checkpoint23'

In [ ]:
# Model XXIII - L2 regularization only on dense layers #

# Model initialization #

classifier23 = Sequential()

# Model architecture #

classifier23.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23.add(Flatten())
classifier23.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', 
                      kernel_regularizer=regularizers.l2(0.01)))
classifier23.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform',
                      kernel_regularizer=regularizers.l2(0.01)))
classifier23.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.01)))

# Model compilation #

classifier23.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath23, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
classifier23.summary()

In [ ]:
# Model training XXIII #

model_L2_23 = classifier23.fit(train_X, train_Y, batch_size=64, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XXIII - the best trained on the validation set #

Model_XXIII = load_model(model_filepath23)
Model_XXIII.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Model XXIIIa filepath #

model_filepath23a = project_dir + '/models/model_checkpoint23a'

In [ ]:
# Model XXIIIa - L2 regularization only on dense layers #

# Model initialization #

classifier23a = Sequential()

# Model architecture #

classifier23a.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23a.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23a.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23a.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid', use_bias=True,
                      bias_initializer='zeros', kernel_initializer='glorot_uniform'))
classifier23a.add(MaxPooling2D((2, 2), strides=(2, 2)))
classifier23a.add(Flatten())
classifier23a.add(Dense(units=256, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform', 
                      kernel_regularizer=regularizers.l2(0.01)))
classifier23a.add(Dense(units=128, activation='relu', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform',
                      kernel_regularizer=regularizers.l2(0.01)))
classifier23a.add(Dense(units=5, activation='softmax', use_bias=True, bias_initializer='zeros', kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.01)))

# Model compilation #

classifier23a.compile(optimizer=optimizers.Adam(0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialization of model training early stop and saving the best model for the time being #

earlystopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
CheckPoint = callbacks.ModelCheckpoint(filepath=model_filepath23a, monitor='val_accuracy', mode='max', verbose=1,
                                       save_best_only=True)

In [ ]:
# Model training XXIIIa #

model_L2_23a = classifier23a.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(validation_X, validation_Y), shuffle=True, 
                            callbacks=[earlystopping, CheckPoint], class_weight={0: 1.14511252939201, 1: 1.50242397531952, 2: 1, 3: 1.72345803842265, 4: 7.04338842975207})

In [ ]:
# Model classification accuracy XXIIIa - the best trained on the validation set #

Model_XXIIIa = load_model(model_filepath23a)
Model_XXIIIa.evaluate(x=validation_X, y=validation_Y)

In [ ]:
# Classification accuracy of the best selected model on the test set #

Model_IV.evaluate(x=test_X, y=test_Y)

In [ ]:
# Comparison of accuracy metrics during the learning process on the training and validation set #

fig8_dir = project_dir + '/Figures/figure8.jpg'
accuracy = model_plain4.history['accuracy']
val_accuracy = model_plain4.history['val_accuracy']

epochs = range(len(accuracy))

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.plot(epochs, accuracy, 'b', linewidth=4, label='Dokładność na zbiorze treningowym')
ax.plot(epochs, val_accuracy, 'r', linewidth=4, label='Dokładność na zbiorze walidacyjnym')
plt.fill_between(epochs, val_accuracy, accuracy, where=np.array(accuracy) > np.array(val_accuracy), color='yellow', label='Overfitting')
ax.scatter(val_accuracy.index(max(val_accuracy)), max(val_accuracy), s=150, c='black')
ax.annotate('val_accuracy max', xy=(39, 0.7), xytext=(30, 0.5), arrowprops=dict(facecolor='black', shrink=0.05))
ax.set_xlabel('Epoka', fontsize=12)
ax.set_ylabel('Dokładność klasyfikacji', fontsize=12)
ax.set_title('Dokładność klasyfikacji względem epoki uczenia modelu', fontsize=13)
ax.legend()
plt.savefig(fig8_dir)

In [ ]:
# Prediction on the test set #

predicted_classes = Model_IV.predict(test_X)
predicted_classes = np.argmax(np.round(predicted_classes), axis=1)

In [ ]:
# Summary of prediction quality for age categories in the test set #

classes = np.unique(test_Y)
target_names =['Kategoria {}'.format(i) for i in classes]
print(classification_report(test_Y, predicted_classes, target_names=target_names))

In [ ]:
# Sketching the multidimensional confusion matrix of the prediction for age categories #

fig9_dir = project_dir + '/Figures/figure9.jpg'
cm = confusion_matrix(test_Y, predicted_classes)

fig, ax = plt.subplots(figsize = (8, 5))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
ax.set_xlabel('Kategoria prognozowana przez model', fontsize=12)
ax.set_ylabel('Prawdziwa kategoria', fontsize=12)
plt.title('Wielowymiarowy confusion matrix', fontsize=16)
plt.yticks(rotation=0)

plt.savefig(fig9_dir)

In [ ]:
# Creating lists of model misclassifications for appropriate age categories #

prediction_error_0_class = []
prediction_error_1_class = []
prediction_error_2_class = []
prediction_error_3_class = []
prediction_error_4_class = []
for i in range(len(test_Y)):
    if test_Y[i] != predicted_classes[i]:
        if test_Y[i] == 0:
            prediction_error_0_class.append((test_X[i], test_Y[i], predicted_classes[i]))
        elif test_Y[i] == 1:
            prediction_error_1_class.append((test_X[i], test_Y[i], predicted_classes[i]))
        elif test_Y[i] == 2:
            prediction_error_2_class.append((test_X[i], test_Y[i], predicted_classes[i]))
        elif test_Y[i] == 3:
            prediction_error_3_class.append((test_X[i], test_Y[i], predicted_classes[i]))
        elif test_Y[i] == 4:
            prediction_error_4_class.append((test_X[i], test_Y[i], predicted_classes[i]))

In [ ]:
# Classification errors for category 0 # 

fig10_dir = project_dir + '/Figures/figure10.jpg'
plt.figure(figsize=[18, 7])

Category_0 = list(zip ([1, 2, 3, 4, 5], [prediction_error_0_class[75], prediction_error_0_class[40],
                                            prediction_error_0_class[67], prediction_error_0_class[17]]))

for i in Category_0:
    plt.subplot(1, 5, i[0])
    plt.imshow(i[1][0])
    plt.title('Prognozowana kategoria:' + str(i[1][2]))
plt.savefig(fig10_dir)   

In [ ]:
# Classification errors for category 1 # 

fig12_dir = project_dir + '/Figures/figure12.jpg'
plt.figure(figsize=[18, 7])

Category_1 = list(zip ([1, 2, 3, 4, 5], [prediction_error_1_class[82], prediction_error_1_class[21],
                                            prediction_error_1_class[64], prediction_error_1_class[47]]))

for i in Category_1:
    plt.subplot(1, 5, i[0])
    plt.imshow(i[1][0])
    plt.title('Prognozowana kategoria:' + str(i[1][2]))
plt.savefig(fig12_dir)   

In [ ]:
## Classification errors for category 2 # 

fig14_dir = project_dir + '/Figures/figure14.jpg'
plt.figure(figsize=[18, 7])

Category_2 = list(zip ([1, 2, 3, 4, 5], [prediction_error_2_class[32], prediction_error_2_class[4],
                                            prediction_error_2_class[87], prediction_error_2_class[39]]))

for i in Category_2:
    plt.subplot(1, 5, i[0])
    plt.imshow(i[1][0])
    plt.title('Prognozowana kategoria:' + str(i[1][2]))
plt.savefig(fig14_dir)   

In [ ]:
## Classification errors for category 3 # 

fig16_dir = project_dir + '/Figures/figure16.jpg'
plt.figure(figsize=[18, 7])

Category_3 = list(zip ([1, 2, 3, 4, 5], [prediction_error_3_class[18], prediction_error_3_class[48],
                                            prediction_error_3_class[1], prediction_error_3_class[43]]))

for i in Category_3:
    plt.subplot(1, 5, i[0])
    plt.imshow(i[1][0])
    plt.title('Prognozowana kategoria:' + str(i[1][2]))
plt.savefig(fig16_dir)   

In [ ]:
## Classification errors for category 4 # 

fig18_dir = project_dir + '/Figures/figure18.jpg'
plt.figure(figsize=[18, 7])

Category_4 = list(zip ([1, 2, 3, 4, 5], [prediction_error_4_class[53], prediction_error_4_class[25],
                                            prediction_error_4_class[1], prediction_error_4_class[87]]))

for i in Category_4:
    plt.subplot(1, 5, i[0])
    plt.imshow(i[1][0])
    plt.title('Prognozowana kategoria:' + str(i[1][2]))
plt.savefig(fig18_dir)   